# Faiss 
参考内容：https://zhuanlan.zhihu.com/p/357414033

In [2]:
import numpy as np

## 第一步创建向量

In [15]:

d = 64                                           # 向量维度
nb = 100000                                      # index向量库的数据量
nq = 10000                                       # 待检索query的数目
np.random.seed(1234)             
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.                # index向量库的向量
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.                # 待检索的query向量
print(xb.shape)

(100000, 64)


## 第二步，构建索引
这里我们选用暴力检索的方法FlatL2，L2代表构建的index采用的相似度度量方法为L2范数，即欧氏距离：

In [10]:
import faiss          
index = faiss.IndexFlatL2(d)             
print(index.is_trained)         # 输出为True，代表该类index不需要训练，只需要add向量进去即可
index.add(xb)                   # 将向量库中的向量加入到index中
print(index.ntotal)             # 输出index中包含的向量总数，为100000 

True
100000


## 第三步，检索TopK相似query：

In [17]:
k = 4                     # topK的K值
D, I = index.search(xq, k)# xq为待检索向量，返回的I为每个待检索query最相似TopK的索引list，D为其对应的距离
print( I[:5])
print(D[-5:])

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[6.53154   6.9787292 7.003937  7.013794 ]
 [4.335266  5.2369385 5.319458  5.7032776]
 [6.0727234 6.576782  6.6139526 6.7323   ]
 [6.6374817 6.6487427 6.8578796 7.009613 ]
 [6.2183533 6.4525146 6.548767  6.5812836]]


## 第三步，检索TopK相似query：
Faiss之所以能加速，是因为它用的检索方式并非精确检索，而是模糊检索。既然是模糊检索，那么必定有所损失，我们用召回率来表示模糊检索相对于精确检索的损失。

在我们实际的工程中，候选向量的数量级、index所占内存的大小、检索所需时间（是离线检索还是在线检索）、index构建时间、检索的召回率等都是我们选择index时常常需要考虑的地方。

首先，我建议关于Faiss的所有索引的构建，都统一使用faiss.index_factory，基本所有的index都支持这种构建索引方法。

以第二章的代码为例，构建index方法和传参方法建议修改为：

In [ ]:
dim, measure = 64, faiss.METRIC_L2
param = 'Flat'
index = faiss.index_factory(dim, param, measure)

三个参数中，dim为向量维数；
最重要的是param参数，它是传入index的参数，代表需要构建什么类型的索引；     
measure为度量方法，目前支持两种，欧氏距离和inner product，即内积。因此，要计算余弦相似度，只需要将vecs归一化后，使用内积度量即可。      
20220102更新，现在faiss官方支持八种度量方式，分别是：  
- METRIC_INNER_PRODUCT（内积）
- METRIC_L1（曼哈顿距离）
- METRIC_L2（欧氏距离）
- METRIC_Linf（无穷范数）
- METRIC_Lp（p范数）
- METRIC_BrayCurtis（BC相异度）
- METRIC_Canberra（兰氏距离/堪培拉距离）
- METRIC_JensenShannon（JS散度）

### Flat ：暴力检索
`优点`：该方法是Faiss所有index中最准确的，召回率最高的方法，没有之一；   
`缺点`：速度慢，占内存大。    
`使用情况`：向量候选集很少，在50万以内，并且内存不紧张。   
`注`：虽然都是暴力检索，faiss的暴力检索速度比一般程序猿自己写的暴力检索要快上不少，所以并不代表其无用武之地，建议有暴力检索需求的同学还是用下faiss。

In [ ]:
dim, measure = 64, faiss.METRIC_L2
param = 'Flat'
index = faiss.index_factory(dim, param, measure)
index.is_trained                                   # 输出为True
index.add(xb)                                      # 向index中添加向量

### IVFx Flat ：倒排暴力检索
`优点`：IVF主要利用倒排的思想，在文档检索场景下的倒排技术是指，一个kw后面挂上很多个包含该词的doc，由于kw数量远远小于doc，因此会大大减少了检索的时间。在向量中如何使用倒排呢？可以拿出每个聚类中心下的向量ID，每个中心ID后面挂上一堆非中心向量，每次查询向量的时候找到最近的几个中心ID，分别搜索这几个中心下的非中心向量。通过减小搜索范围，提升搜索效率。   
`缺点`：速度也还不是很快。     
`使用情况`：相比Flat会大大增加检索的速度，建议百万级别向量可以使用。    
`参数`：IVFx中的x是k-means聚类中心的个数   

In [ ]:
dim, measure = 64, faiss.METRIC_L2 
param = 'IVF100,Flat'                           # 代表k-means聚类中心为100,   
index = faiss.index_factory(dim, param, measure)
print(index.is_trained)                          # 此时输出为False，因为倒排索引需要训练k-means，
index.train(xb)                                  # 因此需要先训练index，再add向量
index.add(xb)  

### PQx ：乘积量化

优点：利用乘积量化的方法，改进了普通检索，将一个向量的维度切成x段，每段分别进行检索，每段向量的检索结果取交集后得出最后的TopK。因为PQ算法把原来的向量空间分解为若干个低维向量空间的笛卡尔积，并对分解得到的低维向量空间分别做量化（quantization）。这样每个向量就能由多个低维空间的量化code组合表示，因此速度很快，而且占用内存较小，召回效果也还可以。  

缺点：召回率相较于暴力检索，下降较多。      

使用情况：内存及其稀缺，并且需要较快的检索速度，不那么在意召回率

参数：PQx中的x为将向量切分的段数，因此，x需要能被向量维度整除，且x越大，切分越细致，时间复杂度越高  

In [ ]:
dim, measure = 64, faiss.METRIC_L2 
param =  'PQ16' 
index = faiss.index_factory(dim, param, measure)
print(index.is_trained)                          # 此时输出为False，因为倒排索引需要训练k-means，
index.train(xb)                                  # 因此需要先训练index，再add向量
index.add(xb)      

### IVFxPQy 倒排乘积量化

优点：工业界大量使用此方法，各项指标都均可以接受，利用乘积量化的方法，改进了IVF的k-means，将一个向量的维度切成x段，每段分别进行k-means再检索。

缺点：集百家之长，自然也集百家之短

使用情况：一般来说，超大规模数据的情况下，各方面没啥特殊的极端要求的话，最推荐使用该方法！

参数：IVFx，PQy，其中的x和y同上

In [ ]:
dim, measure = 64, faiss.METRIC_L2  
param =  'IVF100,PQ16'
index = faiss.index_factory(dim, param, measure) 
print(index.is_trained)                          # 此时输出为False，因为倒排索引需要训练k-means， 
index.train(xb)      

###  LSH 局部敏感哈希

原理：哈希对大家再熟悉不过，向量也可以采用哈希来加速查找，我们这里说的哈希指的是局部敏感哈希（Locality Sensitive Hashing，LSH），不同于传统哈希尽量不产生碰撞，局部敏感哈希依赖碰撞来查找近邻。高维空间的两点若距离很近，那么设计一种哈希函数对这两点进行哈希计算后分桶，使得他们哈希分桶值有很大的概率是一样的，若两点之间的距离较远，则他们哈希分桶值相同的概率会很小。

优点：训练非常快，支持分批导入，index占内存很小，检索也比较快

缺点：召回率非常拉垮。

使用情况：候选向量库非常大，离线检索，内存资源比较稀缺的情况

In [ ]:
dim, measure = 64, faiss.METRIC_L2  
param =  'LSH'
index = faiss.index_factory(dim, param, measure) 
print(index.is_trained)                          # 此时输出为True
index.add(xb)      

### HNSWx （最重要的放在最后说）

优点：该方法为基于图检索的改进方法，检索速度极快，10亿级别秒出检索结果，而且召回率几乎可以媲美Flat，最高能达到惊人的97%。检索的时间复杂度为loglogn，几乎可以无视候选向量的量级了。并且支持分批导入，极其适合线上任务，毫秒级别体验。
缺点：构建索引极慢，占用内存极大（是Faiss中最大的，大于原向量占用的内存大小）

参数：HNSWx中的x为构建图时每个点最多连接多少个节点，x越大，构图越复杂，查询越精确，当然构建index时间也就越慢，x取4~64中的任何一个整数。

使用情况：不在乎内存，并且有充裕的时间来构建index

In [ ]:
dim, measure = 64, faiss.METRIC_L2   
param =  'HNSW64' 
index = faiss.index_factory(dim, param, measure)  
print(index.is_trained)                          # 此时输出为True 
index.add(xb)